Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

---

In [1]:
# AZA MANAMPY CODE ATO FA MNAOVA CELLULE VAOVAO

from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.linear_model import HuberRegressor
from sklearn.datasets import load_boston, load_diabetes, load_iris, load_digits
from scipy.special import huber
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

def grad_check_sparse(f, x, analytic_grad, num_checks=12, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Robust linear regression - Huber loss

In [2]:
data = load_boston()
X_train1, y_train1 = data.data, data.target
w1 = np.random.randn(X_train1.shape[1]) * 0.0001
b1 = np.random.randn(1) * 0.0001

In [3]:
def huber_loss_naive(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations
    
    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias 
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels 
    - epsilon: float
    - alpha: regularization
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0.0
    
    # YOUR CODE HERE
    
    for i in range(len(y)):
        r = 0
        for j in range(len(w)):
            r = r + w[j] * X[i][j]
        r = r + b - y[i]    
        dwhr = np.zeros_like(w)
        if abs(r) <= epsilon:
            hr = (r**2)/2
            for j in range(len(w)):
                dwhr[j] = dwhr[j] + X[i][j] * r
            dbhr = r
        else:
            hr = epsilon * np.abs(r) - (epsilon**2)/2
            for j in range(len(w)):
                dwhr[j] = np.sign(r) * epsilon * X[i][j]
            dbhr = np.sign(r) * epsilon
        loss = loss + hr
        for j in range(len(w)):
            dw[j] = dw[j] + dwhr[j]
        db = db + dbhr
     
    loss = loss/len(y)
    db = db/len(y)
    
    for k in range(len(w)):
        loss = loss + alpha * w[k]**2
        dw[k] = (dw[k]/len(y)) + 2 * alpha * w[k]
        
    return loss, dw, np.array(db).reshape(1,)

## without regularization

In [6]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -12.891700 analytic: -12.891700, relative error: 5.591879e-11
numerical: -12.891700 analytic: -12.891700, relative error: 5.591879e-11
numerical: -5.123308 analytic: -5.123308, relative error: 2.086853e-10
numerical: -17.081635 analytic: -17.081635, relative error: 3.609663e-11
numerical: -0.748838 analytic: -0.748838, relative error: 6.928524e-10
numerical: -12.891700 analytic: -12.891700, relative error: 5.591879e-11
numerical: -24.914970 analytic: -24.914970, relative error: 2.665323e-11
numerical: -24.914970 analytic: -24.914970, relative error: 2.665323e-11
numerical: -481.509943 analytic: -481.509943, relative error: 2.335195e-12
numerical: -92.576117 analytic: -92.576117, relative error: 1.524991e-12
numerical: -0.748838 analytic: -0.748838, relative error: 6.928524e-10
numerical: -12.891700 analytic: -12.891700, relative error: 5.591879e-11
numerical: -5.123308 analytic: -5.123308, relative error: 2.086853e-10
numerical: -481.509943 analytic: -481.50

 ## with regularization

In [11]:
loss, dw1, db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -24.915057 analytic: -24.915057, relative error: 2.462076e-11
numerical: -17.081273 analytic: -17.081273, relative error: 3.929678e-11
numerical: -92.575924 analytic: -92.575924, relative error: 1.322828e-12
numerical: -17.081273 analytic: -17.081273, relative error: 3.929678e-11
numerical: -551.120067 analytic: -551.120067, relative error: 6.506174e-13
numerical: -15.034579 analytic: -15.034579, relative error: 4.748505e-12
numerical: -92.575924 analytic: -92.575924, relative error: 1.322828e-12
numerical: -24.915057 analytic: -24.915057, relative error: 2.462076e-11
numerical: -92.575924 analytic: -92.575924, relative error: 1.322828e-12
numerical: -15.341323 analytic: -15.341323, relative error: 4.011395e-12
numerical: -24.915057 analytic: -24.915057, relative error: 2.462076e-11
numerical: -0.749049 analytic: -0.749049, relative error: 7.191264e-10
numerical: -15.034579 analytic: -15.034579, relative error: 4.748505e-12
numerical: -5.123189 analytic: -5.

In [12]:
def check_inegality(r, epsilon):
    if np.abs(r) <= epsilon:
        return r
    else:
        return epsilon * np.sign(r)
check_inegality_vector = np.vectorize(check_inegality)

In [13]:
def huber_loss_vectorized(w, b, X, y, epsilon=1.35, alpha=0.0001):
    """
    Huber loss for all observations
    
    Inputs:
    - w: array of shape (D,) containing weights
    - b: float bias 
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels 
    - epsilon: float
    - alpha: regularization
    """
    loss = 0.0
    dw = np.zeros_like(w)
    db = 0
    
    # YOUR CODE HERE
    N = X.shape[0]
    r = (X @ w + b) - y
    
    # Loss
    loss = np.sum(huber(epsilon, r)) / N + alpha * (w.T @ w)    
    
    # Dw
    dw_vector = check_inegality_vector(r, epsilon)
    dw = (1/N) * (X.T @ dw_vector) + 2 * alpha * w

    # Db
    db_elements = np.zeros_like(y)
    db_element_inf = r
    db_element_sup = epsilon * np.sign(r)
    db_elements[np.abs(r) <= epsilon] = db_element_inf[np.abs(r) <= epsilon]
    db_elements[np.abs(r) > epsilon] = db_element_sup[np.abs(r) > epsilon]
    db = (1/N) * sum(db_elements)
        
    return loss, dw, np.array(db).reshape(1,) 

## without regularization

In [14]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=0)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -5.123308 analytic: -5.123308, relative error: 1.798882e-11
numerical: -12.891700 analytic: -12.891700, relative error: 6.085666e-12
numerical: -92.576117 analytic: -92.576117, relative error: 5.658178e-13
numerical: -24.914970 analytic: -24.914970, relative error: 1.864054e-12
numerical: -17.081635 analytic: -17.081635, relative error: 3.005375e-13
numerical: -8.484256 analytic: -8.484256, relative error: 4.292105e-15
numerical: -0.748838 analytic: -0.748838, relative error: 9.981585e-11
numerical: -551.120158 analytic: -551.120158, relative error: 1.622417e-13
numerical: -4.878257 analytic: -4.878257, relative error: 1.315611e-11
numerical: -8.484256 analytic: -8.484256, relative error: 4.292105e-15
numerical: -15.340909 analytic: -15.340909, relative error: 7.650677e-12
numerical: -92.576117 analytic: -92.576117, relative error: 5.658178e-13
numerical: -481.509943 analytic: -481.509943, relative error: 6.274487e-14
numerical: -15.340909 analytic: -15.3409

## with regularization

In [15]:
loss, dw1, db1 = huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)

print("Gradient check w")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, dw1, 15, error=1e-9)

print("Gradient check bias")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=1.35, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, db1, 15, error=1e-9)


# Large epsilon
large_eps_loss, large_eps_dw1, large_eps_db1 = huber_loss_naive(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)

print("Gradient check w large epsilon")
# Check with numerical gradient w
f = lambda w1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f, w1, large_eps_dw1, 15, error=1e-9)

print("Gradient check bias large epsilon")
# Check with numerical gradient b
f2 = lambda b1: huber_loss_vectorized(w1, b1, X_train1, y_train1, epsilon=135, alpha=1)[0]
grad_numerical = grad_check_sparse(f2, b1, large_eps_db1, 15, error=1e-9)

Gradient check w
numerical: -12.891911 analytic: -12.891911, relative error: 3.832724e-12
numerical: -12.891911 analytic: -12.891911, relative error: 3.832724e-12
numerical: -8.484183 analytic: -8.484183, relative error: 6.397385e-13
numerical: -0.093220 analytic: -0.093220, relative error: 9.129167e-11
numerical: -24.915057 analytic: -24.915057, relative error: 3.896426e-12
numerical: -15.034579 analytic: -15.034579, relative error: 4.752286e-12
numerical: -551.120067 analytic: -551.120067, relative error: 1.545062e-13
numerical: -12.891911 analytic: -12.891911, relative error: 3.832724e-12
numerical: -4.878487 analytic: -4.878487, relative error: 1.777363e-11
numerical: -0.093220 analytic: -0.093220, relative error: 9.129167e-11
numerical: -551.120067 analytic: -551.120067, relative error: 1.545062e-13
numerical: -24.915057 analytic: -24.915057, relative error: 3.896426e-12
numerical: -481.510064 analytic: -481.510064, relative error: 1.648012e-13
numerical: -4.878487 analytic: -4.87

In [16]:
class LinearModel():
    def __init__(self):
        self.w = None
        self.b = None

    def train(self, X, y, learning_rate=1e-3, alpha=0.0001, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            choice = np.random.choice(N, batch_size, replace=False)

            X_batch = X[choice, :]
            y_batch = y[choice ]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update                                                                
            # Update the weights w and bias b using the gradient and the learning rate.          
            # YOUR CODE HERE
            self.w = self.w - learning_rate * dw
            self.b = self.b - learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class HuberRegression(LinearModel):
    """ Linear regression """

    def loss(self, X_batch, y_batch, alpha):
        return huber_loss_vectorized(self.w, self.b, X_batch, y_batch, alpha=alpha)
    
    def predict(self, X):
        # YOUR CODE HERE
        return (X @ self.w + self.b)

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)

sk_model = HuberRegressor(fit_intercept=True)
sk_model.fit(X_train1, y_train1)
sk_pred = sk_model.predict(X_train1)
sk_mse = mean_squared_error(sk_pred, y_train1)

model = HuberRegression()
model.train(X_train1, y_train1, num_iters=75000, batch_size=64, learning_rate=1e-2, verbose=True)
pred = model.predict(X_train1)
mse = mean_squared_error(pred, y_train1)

print("MSE scikit-learn:", sk_mse)
print("MSE gradient descent model :", mse)
assert mse - sk_mse < 1

iteration 0 / 75000: loss 27.499980
iteration 10000 / 75000: loss 2.888160
iteration 20000 / 75000: loss 4.148812
iteration 30000 / 75000: loss 4.325756
iteration 40000 / 75000: loss 3.159001
iteration 50000 / 75000: loss 3.082888
iteration 60000 / 75000: loss 3.121949
iteration 70000 / 75000: loss 3.574847
MSE scikit-learn: 24.04102301055777
MSE gradient descent model : 24.399431867035325


# Multinomial logistic regression

In [18]:
data = load_iris()
X_train2, y_train2 = data.data, data.target

W = np.random.randn(X_train2.shape[1], 3) * 0.0001

In [19]:
from scipy.special import logsumexp, softmax

def softmax_loss_naive(W, X, y, alpha):
    """
    Softmax loss function WITH FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    
    # Initialization
    loss = 0.0
    dW = np.zeros_like(W)
    
    # Tandremo ny numeric instability
    # YOUR CODE HERE
    N = y.shape[0]
    D = W.shape[0]
    C = W.shape[1]
    
    # Loss
    
     # The term without alpha
    for i in range(N):
        z = X[i] @ W
        loss = loss + z[y[i]] - logsumexp(z)
    
     # The term with alpha
    alpha_term = 0
    for j in range(D):
        for k in range(C):
            alpha_term = alpha_term + W[j][k]**2
    loss = (-1/N) * loss + alpha * alpha_term
    
    # Dw
    
    # Softmax derivative (https://towardsdatascience.com/derivative-of-the-softmax-function-and-the-categorical-cross-entropy-loss-ffceefc081d1)
    
    # The matrix where 1 if (y[i] = C), else 0
    matrix = np.zeros((N, C))
    for i in range(N):
        matrix[i][y[i]] = 1
    
    # The DW adding the term with alpha
    softmax_derivative = matrix - softmax(X @ W, axis=1)
    for j in range(D):
        dW[j] = (-1/N) * X.T[j] @ softmax_derivative + 2 * alpha * W[j]    
    
    return loss, dW

## Without regularization

In [20]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: 0.028050 analytic: 0.028050, relative error: 8.881253e-10
numerical: 0.278359 analytic: 0.278359, relative error: 5.247049e-10
numerical: 0.095894 analytic: 0.095894, relative error: 8.713755e-11
numerical: 0.028050 analytic: 0.028050, relative error: 8.881253e-10
numerical: -0.597786 analytic: -0.597786, relative error: 7.687603e-11
numerical: 0.095894 analytic: 0.095894, relative error: 8.713755e-11
numerical: 0.095894 analytic: 0.095894, relative error: 8.713755e-11
numerical: 0.278359 analytic: 0.278359, relative error: 5.247049e-10
numerical: 0.028050 analytic: 0.028050, relative error: 8.881253e-10
numerical: 0.095894 analytic: 0.095894, relative error: 8.713755e-11
numerical: 0.028050 analytic: 0.028050, relative error: 8.881253e-10
numerical: 0.278359 analytic: 0.278359, relative error: 5.247049e-10


## With regularization

In [21]:
loss, dW = softmax_loss_naive(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_naive(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.275405 analytic: -0.275405, relative error: 2.706203e-11
numerical: -0.598313 analytic: -0.598313, relative error: 7.832539e-11
numerical: -0.030454 analytic: -0.030454, relative error: 4.948893e-09
numerical: -0.030454 analytic: -0.030454, relative error: 4.948893e-09
numerical: -0.166792 analytic: -0.166792, relative error: 2.617687e-10
numerical: 0.317685 analytic: 0.317685, relative error: 1.071850e-10
numerical: 0.317685 analytic: 0.317685, relative error: 1.071850e-10
numerical: 0.095479 analytic: 0.095479, relative error: 9.149837e-11
numerical: -0.247387 analytic: -0.247387, relative error: 5.402637e-10
numerical: 0.095479 analytic: 0.095479, relative error: 9.149837e-11
numerical: 0.028362 analytic: 0.028362, relative error: 7.105147e-10
numerical: -0.030454 analytic: -0.030454, relative error: 4.948893e-09


In [22]:
def softmax_loss_vectorized(W, X, y, alpha, fit_intercept=False):
    """
    Softmax loss function WITHOUT FOR LOOPS

    Inputs:
    - W: array of shape (D, C) containing weights
    - X: array of shape (N, D) containing a minibatch of data
    - y: array of shape (N,) containing training labels
    - alpha: (float) regularization 

    Returns a tuple of:
    - loss as single float
    - gradient with respect to weights W;  same shape as W
    """
    # Initialize the loss and gradient to zero.
    loss = 0.0
    dW = np.zeros_like(W)

    # YOUR CODE HERE
    N = X.shape[1]
    D = X.shape[0]
    
    # https://madalinabuzau.github.io/2016/11/29/gradient-descent-on-a-softmax-cross-entropy-cost-function.html
    
    # Loss
    z = X @ W
    exp_scores = np.exp(z - np.max(z, axis=1, keepdims=True))
        # Softmax activation
    probs = exp_scores/np.sum(exp_scores, axis=1, keepdims=True)
        # Log loss of the correct class of each of our samples
    correct_logprobs = -np.log(probs[np.arange(D), y])
        # Compute the average loss
    loss = np.sum(correct_logprobs)/D
        # Add regularization using the L2 norm
    reg_loss = alpha*np.sum(W*W)
    loss = loss + reg_loss

    # DW
        # Gradient of the loss with respect to scores
    dscores = probs.copy()
        # Substract 1 from the scores of the correct class
    dscores[np.arange(D),y] -= 1
        # samples it's easier to divide dscores beforehand
    dscores /= D
    dW = X.T.dot(dscores)
    # Add gradient regularization 
    dW = dW + 2*alpha*W

    return loss, dW

## Without regularization

In [23]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 0.0)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.042121 analytic: -0.042121, relative error: 2.257001e-11
numerical: 0.028050 analytic: 0.028050, relative error: 8.881257e-10
numerical: 0.317618 analytic: 0.317618, relative error: 2.482308e-12
numerical: 0.278359 analytic: 0.278359, relative error: 4.648795e-10
numerical: 0.095894 analytic: 0.095894, relative error: 3.180781e-10
numerical: 0.317618 analytic: 0.317618, relative error: 2.482308e-12
numerical: 0.278359 analytic: 0.278359, relative error: 4.648795e-10
numerical: -0.247769 analytic: -0.247769, relative error: 5.248354e-10
numerical: -0.597786 analytic: -0.597786, relative error: 7.687613e-11
numerical: -0.030589 analytic: -0.030589, relative error: 4.584582e-09
numerical: 0.028050 analytic: 0.028050, relative error: 8.881257e-10
numerical: -0.597786 analytic: -0.597786, relative error: 7.687613e-11


## With regularization

In [24]:
loss, dW = softmax_loss_vectorized(W, X_train2, y_train2, 2)

f = lambda W: softmax_loss_vectorized(W, X_train2, y_train2, 2)[0]
grad_numerical = grad_check_sparse(f, W, dW, error=1e-7)

numerical: -0.041842 analytic: -0.041842, relative error: 9.646235e-11
numerical: -0.041842 analytic: -0.041842, relative error: 9.646235e-11
numerical: -0.124118 analytic: -0.124118, relative error: 1.348327e-10
numerical: -0.598313 analytic: -0.598313, relative error: 7.832549e-11
numerical: -0.124118 analytic: -0.124118, relative error: 1.348327e-10
numerical: -0.041842 analytic: -0.041842, relative error: 9.646235e-11
numerical: -0.598313 analytic: -0.598313, relative error: 7.832549e-11
numerical: 0.028362 analytic: 0.028362, relative error: 7.105151e-10
numerical: -0.030454 analytic: -0.030454, relative error: 4.766613e-09
numerical: -0.247387 analytic: -0.247387, relative error: 5.402642e-10
numerical: -0.166792 analytic: -0.166792, relative error: 3.283318e-10
numerical: -0.247387 analytic: -0.247387, relative error: 5.402642e-10


## Gradient descent

In [25]:
class LinearModel():
    def __init__(self, fit_intercept=True):
        self.W = None
        self.fit_intercept = fit_intercept

    def train(self, X, y, learning_rate=1e-3, alpha=0, num_iters=100, batch_size=200, verbose=False):
        if self.fit_intercept:
            # YOUR CODE HERE
            temp = np.ones((X.shape[0], 1))
            X = np.concatenate((temp, X), axis=1)
           
        N = X.shape[0] 
        D = X.shape[1]
        
        C = np.max(y) + 1
        self.W = 0.001 * np.random.randn(D, C)

        # Run stochastic gradient descent to optimize W
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            choice = np.random.choice(N, batch_size, replace=False)

            X_batch = X[choice, :]
            y_batch = y[choice]
            
            # evaluate loss and gradient
            loss, dW = self.loss(X_batch, y_batch, alpha)
            loss_history.append(loss)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.          
            # YOUR CODE HERE
            self.W = self.W - learning_rate * dW
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
                
        return loss_history

    def predict(self, X):
        pass

    def loss(self, X_batch, y_batch, reg):
        pass

class MultinomialLogisticRegressor(LinearModel):
    """ Softmax regression """

    def loss(self, X_batch, y_batch, alpha):
        return softmax_loss_vectorized(self.W, X_batch, y_batch, alpha)
    
    def predict(self, X):
        """ 
        Inputs:
        - X: array of shape (N, D) 

        Returns:
        - y_pred: 1-dimensional array of length N, each element is an integer giving the predicted class 
        """
        # YOUR CODE HERE
        if self.fit_intercept:
            # YOUR CODE HERE
            temp = np.ones((X.shape[0], 1))
            X = np.concatenate((temp, X), axis=1)
        
        y_pred = []
        
        # Softmax
        for i in range(X.shape[0]):
            z = X[i] @ self.W
            expo_scores = np.exp(z - np.max(z))
            
            # exp(z - z(max))/sum(exp(z - z(max)))
            results = expo_scores / np.sum(np.exp(z - np.max(z)))
            k = np.argmax(results)
            y_pred.append(k)
        return y_pred

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)

sk_model = LogisticRegression(fit_intercept=False)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=False)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.01

iteration 0 / 75000: loss 1.096879
iteration 10000 / 75000: loss 0.392187
iteration 20000 / 75000: loss 0.316024
iteration 30000 / 75000: loss 0.380803
iteration 40000 / 75000: loss 0.326009
iteration 50000 / 75000: loss 0.333975
iteration 60000 / 75000: loss 0.265954
iteration 70000 / 75000: loss 0.258352
Accuracy scikit-learn: 0.86
Accuracy gradient descent model : 0.8666666666666667


In [27]:
sk_model = LogisticRegression(fit_intercept=True)
sk_model.fit(X_train2, y_train2)
sk_pred = sk_model.predict(X_train2)
sk_accuracy = accuracy_score(y_train2, sk_pred)

model = MultinomialLogisticRegressor(fit_intercept=True)
model.train(X_train2, y_train2, num_iters=75000, batch_size=64, learning_rate=1e-3, verbose=True)
pred = model.predict(X_train2)
model_accuracy = accuracy_score(y_train2, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 0.02

iteration 0 / 75000: loss 1.098294
iteration 10000 / 75000: loss 0.334847
iteration 20000 / 75000: loss 0.267632
iteration 30000 / 75000: loss 0.236719
iteration 40000 / 75000: loss 0.213925
iteration 50000 / 75000: loss 0.147083
iteration 60000 / 75000: loss 0.192649
iteration 70000 / 75000: loss 0.134629
Accuracy scikit-learn: 0.9733333333333334
Accuracy gradient descent model : 0.96


# K-Nearest Neighbor

## Computing distances

In [28]:
data = load_digits()
X_train3, y_train3 = data.data, data.target
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_train3, y_train3, test_size=0.33, random_state=2)

def get_distances_two_loops_with_norm(X_train, X_test):
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            distances[i, j] = np.linalg.norm(X_test[i] - X_train[j])
    return distances

In [29]:
def get_distances_two_loops(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        for j in range(num_train):
            # Ataovy ao anaty distances[i, j] ny distance entre ith test point sy th training point
            # Aza manao boucle instony ato anatiny
            # TSY MAHAZO MAMPIASA np.linalg.norm() :D
            # YOUR CODE HERE
            distances[i, j] = np.sqrt((X_test[i] - X_train[j]).T @ (X_test[i] - X_train[j]))
    return distances

In [30]:
distances = get_distances_two_loops(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


In [31]:
def compute_distances_one_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - dists: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))
    for i in range(num_test):
        # Ataovy ao anaty dists[i, j] ny distance entre ith test point sy th training point
        # Aza manao boucle instony ato anatiny
        # TSY MAHAZO MAMPIASA np.linalg.norm() :D
            
        # YOUR CODE HERE
        distances[i] = np.sqrt(np.sum((X_train - X_test[i])**2, axis = 1))
    return distances    

In [32]:
distances = compute_distances_one_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


In [33]:
def get_distances_zero_loop(X_train, X_test):
    """
    Compute the distance between each test point in X_test and each training point
    in X_train 

    Inputs:
    - X_test: array of shape (num_test, D) 

    Returns:
    - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
    the ith test point and the jth training point.
    """
    num_test = X_test.shape[0]
    num_train = X_train.shape[0]
    distances = np.zeros((num_test, num_train))  
    # Ataovy ao anaty dists[i, j] ny distance entre ith test point sy th training point
    # Aza manao boucle instony
    # TSY MAHAZO MAMPIASA np.linalg.norm() NA FONCTIONS AO AMIN'NY SCIPY :D
            
    # YOUR CODE HERE
    
    # Using the formula (a - b)**2 = a**2 - 2*a*b + b**2 (https://medium.com/@souravdey/l2-distance-matrix-vectorization-trick-26aa3247ac6c)
    distances = np.sum((X_train)**2, axis = 1) - 2 * X_test @ X_train.T + np.sum((X_test)**2, axis = 1)[:, np.newaxis]
    distances = np.sqrt(distances)
    
    return distances

In [34]:
distances = get_distances_zero_loop(X_train3, X_test3)
true_distances = get_distances_two_loops_with_norm(X_train3, X_test3)

difference = np.linalg.norm(distances - true_distances, ord='fro')

print(difference)
assert difference < 1e-10

0.0


## K-Nearest Neighbor (knn) classifier

In [35]:
from scipy.stats import mode

class KNearestNeighborClassifier():
    """ kNN classifier using L2 distance """

    def __init__(self, k=1):
        """
        Inputs:
        - k: number of nearest neighbors that vote for the predicted labels.
        """
        self.k = k

    def fit(self, X, y):
        """
        Train the classifier. Just memorize the training data.

        Inputs:
        - X: array of shape (num_train, D) 
        - y: array of shape (N,) 
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        """
        Predict labels for test data using this classifier.

        Inputs:
        - X: array of shape (num_test, D) 

        Returns:
        - y: array of shape (num_test,) 
        """
        distances = get_distances_zero_loop(self.X_train, X)
        return self.predict_labels(distances)

    def predict_labels(self, distances):
        """
        Given a matrix of distances between test points and training points,
        predict a label for each test point.

        Inputs:
        - distances: array of shape (num_test, num_train), dists[i, j] is Euclidean distance between 
        the ith test point and the jth training point.

        Returns:
        - y:  array of shape (num_test,) 
        """
        num_test = distances.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            # list storing the labels of the k nearest neighbors to the ith test point.
            closest_y = []

            # Ampidirina ao anaty closest_y ny labels an'ny k neighbors akaiky indrindra
            # Jereo fampiasana np.argsort
            # YOUR CODE HERE
            indice_nearest = np.argsort(distances[i])
            closest_y = self.y_train[indice_nearest[:self.k]]

            # Tadiavo ny label betsaka indrindra dia iny no atao prediction
            # Raha misy mitovy dia izay label kely raisina
            # YOUR CODE HERE
            nearest_y = mode(closest_y)
            y_pred[i] = nearest_y.mode[0]

        return y_pred

In [36]:
sk_model = KNeighborsClassifier(n_neighbors=3)
sk_model.fit(X_train3, y_train3)
sk_pred = sk_model.predict(X_test3)
sk_accuracy = accuracy_score(y_test3, sk_pred)

model = KNearestNeighborClassifier(k=3)
model.fit(X_train3, y_train3)
pred = model.predict(X_test3)
model_accuracy = accuracy_score(y_test3, pred)

print("Accuracy scikit-learn:", sk_accuracy)
print("Accuracy gradient descent model :", model_accuracy)
assert sk_accuracy - model_accuracy < 1e-10

Accuracy scikit-learn: 0.9831649831649831
Accuracy gradient descent model : 0.9831649831649831


## cross-validation

In [37]:
num_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]

X_train_folds = []
y_train_folds = []

# Split up the data into folds
# X_train_folds and y_train_folds lits of length num_folds

# YOUR CODE HERE
N = X_train3.shape[0]
# Shuffle the data with random permutation and split into num_folds data
indice_nearest = np.array_split(np.random.permutation(N), num_folds)

for i in range(num_folds):
    X_train_folds.append(X_train3[indice_nearest[i]])
    y_train_folds.append(y_train3[indice_nearest[i]])
# Convert to nd.array
X_train_folds = np.array(X_train_folds, dtype = object)
y_train_folds = np.array(y_train_folds, dtype = object)

# A dictionary of length num_folds holding the accuracies for different values of k 
k_to_accuracies = {}

# Ataovy ary ilay k-fold cross validation 
# Atao ao anaty k_to_accuracies ny accuracy isaky ny valeur k
# YOUR CODE HERE
for i in range(num_folds):
    # Test all folds
    X_test_cross = X_train_folds[i]
    y_test_cross = y_train_folds[i]
    
    # Separate the training data
    mask = np.ones((num_folds,), dtype = bool)
    mask[i] = False
    X_train_cross = np.concatenate(X_train_folds[mask])
    y_train_cross = np.concatenate(y_train_folds[mask])
    
    k_to_accuracies[i] = []
    for k in k_choices:
        test = KNearestNeighborClassifier(k=k)
        test.fit(X_train_cross, y_train_cross)
        pred = model.predict(X_test_cross)
        k_to_accuracies[i].append(accuracy_score(y_test_cross, pred))

In [38]:
for k in sorted(k_to_accuracies):
    for accuracy in k_to_accuracies[k]:
        print('k = %d, accuracy = %f' % (k, accuracy))

k = 0, accuracy = 0.987552
k = 0, accuracy = 0.983402
k = 0, accuracy = 0.979253
k = 0, accuracy = 0.975104
k = 0, accuracy = 0.975104
k = 0, accuracy = 0.975104
k = 0, accuracy = 0.975104
k = 0, accuracy = 0.962656
k = 0, accuracy = 0.941909
k = 0, accuracy = 0.908714
k = 1, accuracy = 0.987552
k = 1, accuracy = 0.987552
k = 1, accuracy = 0.991701
k = 1, accuracy = 0.983402
k = 1, accuracy = 0.987552
k = 1, accuracy = 0.983402
k = 1, accuracy = 0.970954
k = 1, accuracy = 0.966805
k = 1, accuracy = 0.933610
k = 1, accuracy = 0.896266
k = 2, accuracy = 0.991701
k = 2, accuracy = 0.995851
k = 2, accuracy = 0.987552
k = 2, accuracy = 0.983402
k = 2, accuracy = 0.983402
k = 2, accuracy = 0.983402
k = 2, accuracy = 0.979253
k = 2, accuracy = 0.975104
k = 2, accuracy = 0.941909
k = 2, accuracy = 0.925311
k = 3, accuracy = 0.983333
k = 3, accuracy = 0.987500
k = 3, accuracy = 0.987500
k = 3, accuracy = 0.991667
k = 3, accuracy = 0.983333
k = 3, accuracy = 0.987500
k = 3, accuracy = 0.983333
k